In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
url = 'https://services.nvd.nist.gov/rest/json/cves/2.0?cpeName=cpe:2.3:o:microsoft:windows_11_21h2:10.0.22000&cvssV3Severity=CRITICAL&resultsPerPage=50&startIndex=0&sourceIdentifier=secure@microsoft.com'
response = requests.get(url)
if response.status_code == 200:
    # Parsear la respuesta JSON
    data = response.json()
    
    # Extraer la lista de vulnerabilidades
    vulnerabilities = data.get('vulnerabilities', [])
    cvssMetrics31 = data.get('cvssMetricV31', [])
    
    # Crear una lista para almacenar los datos
    data_list = []
    
    # Iterar sobre cada vulnerabilidad y extraer la información
    for vuln in vulnerabilities:
        cve_data = vuln.get('cve', {})
        cve_id = cve_data.get('id', '')        
        published = cve_data.get('published', '')
        last_modified = cve_data.get('lastModified', '')
        
        # Extremos la Descripción
        descriptions = cve_data.get('descriptions', [])
        description = descriptions[0].get('value', '') if descriptions else ''
            
        # Extramos datos de Métricas CVSS
        metrics = cve_data.get('metrics', {})
                
        # Extraemos datos de CVSS v3.1
        cvss_v31 = metrics.get('cvssMetricV31', [])
        if cvss_v31:
            cvss_v31_data = cvss_v31[0].get('cvssData', {})
            cvss_v31_score = cvss_v31_data.get('baseScore', '')
            cvss_v31_vector = cvss_v31_data.get('vectorString', '')
            attackComplexity = cvss_v31_data.get('attackComplexity', '')
            privilegesRequired = cvss_v31_data.get('privilegesRequired', '')
            userInteraction = cvss_v31_data.get('userInteraction', '')
            scope = cvss_v31_data.get('scope', '')
            confidentiality = cvss_v31_data.get('confidentialityImpact', '')
            integrity = cvss_v31_data.get('integrityImpact', '')
            availability = cvss_v31_data.get('availabilityImpact', '')            
                        
        else:
            cvss_v31_score = ''
            cvss_v31_vector = ''
        
               # Añadir los datos extraídos a una lista
        data_list.append({
            'CVE_ID': cve_id,
            'Published': published,
            'LastModified': last_modified,
            'Description': description,
            'CVSSv3.1_Score': cvss_v31_score,
            'CVSSv3.1_Vector': cvss_v31_vector,
            'attackComplexity': attackComplexity,
            'privilegesRequired': privilegesRequired,
            'userInteraction': userInteraction,
            'scope': scope,
            'confidentialityImpact': confidentiality,
            'integrityImpact': integrity,
            'availabilityImpact': availability            
        })
    
    # Creamos un DataFrame con pandas
    df = pd.DataFrame(data_list)
    
    # Seleccionamos las primeras 10 filas y 13 columnas
    src = df.iloc[:10, :13]

    print(df.isnull().sum()) #Verificar si hay valores nulos
    
    #Comprobamos el tipo de datos de la columna CVE_ID
    tipo_dato = df['CVE_ID'].dtype
    print(f"El tipo de dato de la columna es: {tipo_dato}")
    tipos_elementos = df['CVE_ID'].apply(type).value_counts()
    print("Tipos de datos en la columna 'CVE_ID':")
    print(tipos_elementos)
    
    # Si queremos mostrar los datos extraídos de NIST descomentar la siguiente línea
    #print(src)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

#******** Llamar al script py de creación de BDD Vulnerabilidades y table NIST_CVEs, aun no funciona
#!python ./Crear_NIST_BDD.py


CVE_ID                   0
Published                0
LastModified             0
Description              0
CVSSv3.1_Score           0
CVSSv3.1_Vector          0
attackComplexity         0
privilegesRequired       0
userInteraction          0
scope                    0
confidentialityImpact    0
integrityImpact          0
availabilityImpact       0
dtype: int64
El tipo de dato de la columna es: object
Tipos de datos en la columna 'CVE_ID':
CVE_ID
<class 'str'>    21
Name: count, dtype: int64
